In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [31]:
train_file = '/data/datasets/visdrone/annotations/visdrone_train_remove_dense_sw640_sh512.json'
train_tmp_file = '/data/datasets/visdrone/annotations/visdrone_train_remove_dense_sw640_sh512_tmp.json'
train_coco = COCO(train_file)

loading annotations into memory...
Done (t=2.39s)
creating index...
index created!


In [18]:
# display COCO categories and supercategories
cats = train_coco.loadCats(train_coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

COCO categories: 
person



In [19]:
with open(train_file) as f:
    train = json.load(f)

In [20]:
train_annotations = train['annotations']
train_images = train['images']

In [21]:
len(train_images)

51008

In [22]:
train['categories'] = [{'name': 'person', 'id': 1}]

In [23]:
removed_images = []
remaining_images = []

In [24]:
keep_annotations = []

In [25]:
for a in train_annotations:
    a['iscrowd'] = 0
    if a['category_id'] == 2:
        a['category_id'] = 1
    if a['category_id'] == 1:
        a['ignore'] = False
        keep_annotations.append(a)
    else:
        a['ignore'] = True

In [26]:
len(keep_annotations)

87107

In [27]:
train['annotations'] = keep_annotations

In [33]:
with open(train_tmp_file, 'w') as f:
    json.dump(train, f)

In [34]:
train_tmp = COCO(train_tmp_file)

loading annotations into memory...
Done (t=1.59s)
creating index...
index created!


In [35]:
for img in train_images:
    found_bbox = False
    anns = train_tmp.loadAnns(train_tmp.getAnnIds(imgIds=[img['id']]))
    for a in anns:
        if not a['ignore']:
            found_bbox = True
            break
    if not found_bbox:
        removed_images.append(img)
    else:
        remaining_images.append(img)

In [36]:
train['images'] = remaining_images

In [37]:
len(removed_images), len(remaining_images)

(35280, 15728)

In [38]:
with open(train_file, 'w') as f:
    json.dump(train, f)

In [39]:
len(train['images'])

15728

In [29]:
train_annotations[52]

{'id': 52,
 'image_id': 0,
 'category_id': 1,
 'bbox': [428.0, 197.0, 8.0, 13.0],
 'score': 1,
 'truncation': 0,
 'occlusion': 1,
 'segmentation': [[428.0, 197.0, 428.0, 210.0, 436.0, 210.0, 436.0, 197.0]],
 'area': 104.0,
 'size': 10.198039027185569,
 'iscrowd': 0,
 'ignore': False}